In [2]:
import os
import pandas as pd
import argparse
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()

# --- Cấu hình kết nối Database ---
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

try:
    print(f"🔗 Kết nối DB: {DB_HOST}:{DB_PORT}/{DB_NAME}")
    engine = create_engine(DATABASE_URL)
    print("✅ Kết nối DB thành công")
except Exception as e:
    engine = None
    print(f"❌ Không tạo được kết nối DB: {e}")

# --- Hàm đọc data từ Database ---
def get_binance_data(symbol, interval):
    """

    """
    print(f"📊 Lấy dữ liệu từ DB: {symbol} {interval}")
    if engine is None:
        print("❌ Chưa có kết nối DB hợp lệ.")
        return pd.DataFrame()

    table_name =  "proddb.coin_prices_1h"

    query = f"""
        SELECT *
        FROM {table_name}
        WHERE symbol = %(symbol)s
          AND open_time >= EXTRACT(EPOCH FROM NOW()) - 90*24*3600 
          AND open_time <= EXTRACT(EPOCH FROM NOW())
        ORDER BY open_time ASC
    """

    try:
        print(f"🔍 Query {table_name}")
        df = pd.read_sql(
            query,
            con=engine,
            params={
                "symbol": symbol,
            },
        )

        if 'open_time' not in df.columns:
            print("❌ Thiếu cột 'open_time' trong dữ liệu trả về từ DB.")
            print(f"   Columns: {list(df.columns)}")
            return pd.DataFrame()

        required_cols = {'open', 'high', 'low', 'close'}
        if not required_cols.issubset(df.columns):
            print(f"❌ Thiếu cột cần thiết: {required_cols - set(df.columns)}")
            return pd.DataFrame()

        df['datetime'] = pd.to_datetime(df['open_time'], unit='s')
        df = df.sort_values('datetime').reset_index(drop=True)

        print(f"✅ Đã đọc {len(df)} dòng từ DB bảng {table_name}")
        if len(df) > 0:
            print(f"   Thời gian: {df['datetime'].min()} đến {df['datetime'].max()}")
            print(f"   Giá cuối: ${df.iloc[-1]['close']:.4f}")

        return df
    except Exception as e:
        print(f"❌ Lỗi khi đọc dữ liệu từ DB: {e}")
        return pd.DataFrame()

# Example usage of get_binance_data in a notebook cell

symbol = "BTCUSDT"
interval = "1h"

df = get_binance_data(symbol, interval)

if not df.empty:
    print(df.head())
else:
    print("Không lấy được dữ liệu cho", symbol, interval)


🔗 Kết nối DB: 34.124.179.109:5432/vistia
✅ Kết nối DB thành công
📊 Lấy dữ liệu từ DB: BTCUSDT 1h
🔍 Query proddb.coin_prices_1h
✅ Đã đọc 2158 dòng từ DB bảng proddb.coin_prices_1h
   Thời gian: 2025-06-10 03:00:00 đến 2025-09-08 01:00:00
   Giá cuối: $110890.5700
   update_time   open_time   symbol       open       high        low  \
0   1756982365  1749524400  BTCUSDT  109650.95  109684.89  109382.38   
1   1756982365  1749528000  BTCUSDT  109568.03  109649.98  109366.00   
2   1756982365  1749531600  BTCUSDT  109528.55  109737.68  109200.00   
3   1756982365  1749535200  BTCUSDT  109310.49  109650.00  109161.00   
4   1756982365  1749538800  BTCUSDT  109502.69  109517.16  109095.67   

       close     volume   quote_asset  num_trades   buy_base     buy_quote  \
0  109568.02  369.14727  4.041644e+07       63357  160.32094  1.755283e+07   
1  109528.55  369.49008  4.044832e+07       52312  197.54344  2.162380e+07   
2  109310.49  529.37516  5.792657e+07       84427  237.49521  2.598570